***En esta entrega, seguiremos trabajando con la API de transporte de la Ciudad de Buenos Aires. Nos enfocaremos en normalizar la información e insertarla en una tabla que disponga de clave primaria compuesta.***

* 
    API Doc: https://api-transporte.buenosaires.gob.ar/console
*   API Console: https://apitransporte.buenosaires.gob.ar/console/


**Librerías a utilizar**

In [1]:
import psycopg2 #Conexión con redshift
import requests #Para conectar con la api
import datetime #Manejo de fechas
import pandas as pd
import os
from dotenv import load_dotenv

**Funcion encargada de conectar con la API**

In [2]:
def consulta_api_params(url,params):
    try:
        response = requests.get(url,params = params)
        if response.status_code == 200:  #Corroboramos que la peticion sea exitosa
            print("Respuesta de la API:")
            print(response.json())  # Convertimos la respuesta a JSON
            return response.json()
        else:
            print("Error al consultar la API. Código de estado:", response.status_code)
    except Exception as e:
        print("Error al conectarse a la API:", e)

**Funcion que nos permite pasarle los diferentes parametros a la consulta.**

In [3]:
def get_params(route_id,agency_id,trip,client_id,client_secret):
    params = {}
    
    if(route_id != ""):
        params['route_id'] = route_id
    
    if(agency_id != ""):
        params['agency_id'] = agency_id
        
    if(trip != ""):
        params['Trip'] = trip
        
    if(client_id != ""):
        params['client_id'] = client_id
        
    if(client_secret != ""):
        params['client_secret'] = client_secret
        
    return params

**Funcion encargada de corregir valores que por defecto vienen con valores incompatibles**

In [4]:
def correcion_valores(docjson):
    df = pd.DataFrame(docjson)
    df['datetime_consulta'] = pd.to_datetime(df['timestamp'],unit='s') - datetime.timedelta(hours=3)
    df['speed'] = df['speed'].round(3)
    df['timestamp'] = df['timestamp'].astype(str)
    df['direction'] = df['direction'].astype(str)
    df['agency_id'] = df['agency_id'].astype(str)
    return df

**Seteamos las credenciales y parametros para la conexión**

In [5]:
url_p = 'https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositionsSimple'
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

params = get_params("","","",client_id,client_secret)

datos_json = consulta_api_params(url_p,params = params)
# Crear DataFrame a partir del JSON
df = correcion_valores(datos_json)



Respuesta de la API:
[{'route_id': '1465', 'latitude': -34.68735, 'longitude': -58.68619, 'speed': 5.277777, 'timestamp': 1713837106, 'id': '1822', 'direction': 1, 'agency_name': 'MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.', 'agency_id': 82, 'route_short_name': '253A', 'tip_id': '95274-1', 'trip_headsign': 'Expreso x Baradero - Olimpo y C. de Cintura'}, {'route_id': '1468', 'latitude': -34.6596, 'longitude': -58.66542, 'speed': 0, 'timestamp': 1713837108, 'id': '1854', 'direction': 0, 'agency_name': 'MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.', 'agency_id': 82, 'route_short_name': '153A', 'tip_id': '95462-1', 'trip_headsign': 'a Cañuelas'}, {'route_id': '1467', 'latitude': -34.68874, 'longitude': -58.66313, 'speed': 5.833333, 'timestamp': 1713837106, 'id': '1855', 'direction': 1, 'agency_name': 'MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.', 'agency_id': 82, 'route_short_name': '321A', 'tip_id': '95371-1', 'trip_headsign': 'a Chacarita x Camarones'}, {'route_id': '1468', 'latitude': -34.6924, 'longitude': -58.6

**Revisamos tipos de datos del dataframe obtenido**

In [6]:
print(df.dtypes)

route_id                     object
latitude                    float64
longitude                   float64
speed                       float64
timestamp                    object
id                           object
direction                    object
agency_name                  object
agency_id                    object
route_short_name             object
tip_id                       object
trip_headsign                object
datetime_consulta    datetime64[ns]
dtype: object


**Reordenamos el Dataframe para su posterior inserción en nuestra tabla en Redshift**

In [7]:
new_order = ['id','datetime_consulta','route_id','latitude','longitude','speed','timestamp','direction','agency_name','agency_id','route_short_name','tip_id','trip_headsign']

df = df[new_order]

print("Dataframe Reordenado")
display(df)

Dataframe Reordenado


,id,datetime_consulta,route_id,latitude,longitude,speed,timestamp,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
0,1822,2024-04-22 22:51:46,1465,-34.68735,-58.686190,5.278,1713837106,1,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,253A,95274-1,Expreso x Baradero - Olimpo y C. de Cintura
1,1854,2024-04-22 22:51:48,1468,-34.65960,-58.665420,0.000,1713837108,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,153A,95462-1,a Cañuelas
2,1855,2024-04-22 22:51:46,1467,-34.68874,-58.663130,5.833,1713837106,1,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,321A,95371-1,a Chacarita x Camarones
3,1856,2024-04-22 22:51:48,1468,-34.69240,-58.664085,5.000,1713837108,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,153A,95460-1,a Cañuelas
4,1882,2024-04-22 22:51:50,316,-34.74574,-58.294520,3.611,1713837110,1,EL NUEVO HALCON S.A.,63,148D,24208-1,a Constitución
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,69593,2024-04-22 21:33:46,835,-34.54149,-58.824200,2.222,1713832426,1,LA PRIMERA DE GRAND BOURG S.A.,132,315E,55030-1,a Est. Lanus
3646,69602,2024-04-22 22:51:48,704,-34.73451,-58.458695,8.333,1713837108,1,CIA ANDRADE EMPRESA DE TRANSPORTE DE PASAJEROS...,96,283B1,47865-1,a Balneario de Quilmes
3647,69649,2024-04-22 22:51:46,288,-34.82592,-58.388700,0.000,1713837106,0,SAN VICENTE S.A. DE TRANSPORTES,68,177B,21345-1,a Est. Bulogne
3648,69749,2024-04-22 22:51:46,1776,-34.65128,-58.502360,8.056,1713837106,0,LA VECINAL DE MATANZA SACI. DE MICROOMNIBUS,76,180A,115565-1,D - Av. Congreso y Bucarelli


**Función encargada de cargar los datos obtenidos en la tabla de Redshift**

In [8]:
def cargar_df_redshift(df):
    try:
        conection = psycopg2.connect(
        dbname = os.getenv('NAME_DATABASE'),
        user = os.getenv('USER_DATABASE'),
        password = os.getenv('PASS_DATABASE'),
        host = os.getenv('HOST_DATABASE'),
        port= os.getenv('PORT_DATABASE'))
        
        
        datetime_insert = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") #Campo de fecha de insercion en la tabla
        cursor = conection.cursor()
        
        for i in range(df.shape[0]-1):
            tupla_final = tuple(df.iloc[i]) + (datetime_insert,)
            cursor.execute("INSERT INTO cristiangen16_coderhouse.Fact_Bus_Position(idTrack, dttmCatch,idRoute ,adLatitude,adLongitude,adSpeed,adTimestamp,adDirection,adAgencyName,idAgency,adRouteShortName,idTransport,adTripHeadsign,dttmInsert) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)" , tupla_final)
        
        conection.commit()
        print("Datos cargados en Amazon Redshift con éxito.")
        cursor.close()
        conection.close()
        
    except Exception as e:
        print("Error al cargar datos en Redshift:", e)

**Invocamos a la función para cargar los datos**

In [9]:
cargar_df_redshift(df)

Datos cargados en Amazon Redshift con éxito.
